# 機械学習の数学 5（重回帰分析）

前章の単回帰分析では入力変数 $x$ として部屋の広さを採用し、出力変数 $y$ として家賃を採用しました。  

しかし、実際の問題を考える場合、入力変数 $x$ は 1 種類では足りません。家賃を予測するためには部屋の広さ以外にも、駅からの距離・築年数・間取りなど様々な要因が影響する為です。このように、機械学習においては現実に起きている問題を説明する為の適切な入力変数の選定が重要となります。  

こういった入力変数を多変数とする場合に重回帰分析を用います。  
部屋の広さを $x_{1}$、駅からの距離を $x_{2}$、…、犯罪発生率を $x_{M}$ といった形で表し、$M$ 個の入力変数を扱うことを考えてみましょう。  

全体の手順は単回帰分析と同様に進んで行きます。  

## 本章の構成
- モデルを決める
- 目的関数を決める
- パラメータを最適化する

## モデルを決める

単回帰分析では次のように直線の方程式をモデルとして用いていました。  

$$
y = wx + b
$$

重回帰分析でも単回帰分析と似た形のモデルを定義します。  
また、総和の記号 $∑$ を使って表記することもできます。  

$$
\begin{aligned}
y 
&= w_{1}x_{1} + w_{2}x_{2} + \cdots + w_{M}x_{M} + b \\
&= \sum_{m=1}^{M} w_{m} x_{m} + b
\end{aligned}
$$


単回帰分析の際は二次元平面を考え、その平面上に存在するデータに最もよく適合する直線の方程式を考えましたが、今回は $M$ 次元空間に存在するデータに最もよく適合する方程式を見つける事になります。  

ここでバイアス $b$ の扱い方を改めて考えます。単回帰分析では、中心化を前処理として施し、バイアス $b$ を省略することで、簡潔に定式化することができました。重回帰分析では、 $M$ 個の重み $w_{1}, w_{2}, \dots, w_{M}$ と 1 個のバイアス $b$ があり、合わせて $M + 1$ 個のパラメータが存在します。これらのパラメータをうまく定式化することを考えます。  

そこで、今回は $x_0 = 1$、$w_0 = b$ とおくことで次のように $b$ を総和の内側の項に含めて、簡潔に表記できるようにします。
（ここで、 $\sum$ 記号の下部が $m=1$ ではなく $m=0$ となっていることに注意してください。）


$$
\begin{aligned}
y
&= w_{1}x_{1} + w_{2}x_{2} + \cdots + w_{M}x_{M} + b \\
&= w_{1}x_{1} + w_{2}x_{2} + \cdots + w_{M}x_{M} + w_{0}x_{0} \\
&= w_{0}x_{0} + w_{1}x_{1} + \cdots + w_{M}x_{M} \\
&= \sum_{m=0}^M w_{m} x_{m}
\end{aligned}
$$


さらに別の方法として、3 章の線形代数で学んだベクトルの内積で表記するとシンプルに記述できます。

$$
\begin{aligned}
y
&= w_{0}x_{0} + w_{1}x_{1} + \cdots + w_{M}x_{M} \\
&=
\begin{bmatrix}
w_{0} & w_{1} & \cdots  & w_{M}
\end{bmatrix}
\begin{bmatrix}
x_{0} \\
x_{1} \\
\vdots \\
x_{M}
\end{bmatrix} \\
&= {\bf w}^{\rm T}{\bf x}
\end{aligned}
$$


このモデルが持つパラメータは前述の通り $M + 1$ 個あり、$M + 1$ 次元のベクトル ${\bf w}$ で表されています。
重回帰分析では、この ${\bf w}$ のすべての要素について最適な値を求めます。

## 目的関数を決める

単回帰分析の例と比べると、入力変数は増えましたが、家賃を目標値としている点は変わっていません。  
そこで、単回帰分析と同じ目的関数である二乗和誤差を用います。  

$$
L = (t_1 - y_1)^2 + (t_2 - y_2)^2 + \cdots + (t_N - y_N)^2
$$

この二乗和誤差はベクトルの内積を用いて表記すると次の様になります。  

$$
\begin{aligned}
L
&= (t_1 - y_1)^2 + (t_2 - y_2)^2 + \cdots + (t_N - y_N)^2 \\
&= \begin{bmatrix}
t_1 - y_1 & t_2 - y_2 & \cdots & t_N - y_N
\end{bmatrix}
\begin{bmatrix}
t_1 - y_1 \\
t_2 - y_2 \\
\vdots \\
t_N - y_N
\end{bmatrix} \\
&= ({\bf t} - {\bf y})^{\rm T}({\bf t} - {\bf y})
\end{aligned}
$$



ここで、内積には交換法則が成り立つため、${\bf w}^{\rm T}{\bf x}$ は ${\bf x}^{\rm T}{\bf w}$ と書くこともできます。これを利用して、モデルの方程式 ${\bf y} = {\bf w}^{\rm T}{\bf x}$ を、以下のように変形します。

$$
\begin{aligned}
{\bf y} =
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_N
\end{bmatrix} =
\begin{bmatrix}
{\bf x}_1^{\rm T}{\bf w} \\
{\bf x}_2^{\rm T}{\bf w} \\
\vdots \\
{\bf x}_N^{\rm T}{\bf w}
\end{bmatrix} =
\begin{bmatrix}
{\bf x}_1^{\rm T} \\
{\bf x}_2^{\rm T} \\
\vdots \\
{\bf x}_N^{\rm T}
\end{bmatrix}
{\bf w}
\end{aligned}
$$

さらに、${\bf x}_n^{\rm T} = \bigl[ x_{n0},\ x_{n1},\  x_{n2},\ \dots,\  x_{nM} \bigr]$ $(n = 1, \dots, N)$ と展開すると、

$$
\begin{aligned}
{\bf y}
&= \begin{bmatrix}
x_{10} & x_{11} & x_{12} & \cdots & x_{1M} \\
x_{20} & x_{21} & x_{22} & \cdots & x_{2M} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_{N0} & x_{N1} & x_{N2} & \cdots & x_{NM}
\end{bmatrix}
\begin{bmatrix}
w_{0} \\
w_{1} \\
w_{2} \\
\vdots \\
w_{M}
\end{bmatrix} \\
&= {\bf X}{\bf w}
\end{aligned}
$$

と表記できます。
ここで、$N \times M$ 行列 ${\bf X}$ は、各行が各データを表しており、各列が各入力変数を表しています。各列はそれぞれ入力変数の種類に対応しており、例えば、部屋の広さや、駅からの距離などです。

各行が表すデータ点がどのように表されているかを説明するため、具体的な数値例を挙げます。
部屋の広さ $= 50{\rm m}^2$ 、駅からの距離 $= 600 {\rm m}$ 、犯罪発生率 $= 2\%$ という 3 つの入力変数を考える場合、$M = 3$ であり、これが $n$ 個目のデータのとき、${\bf x}_n^{\rm T}$ は、

$$
{\bf x}_n^{\rm T} =
\begin{bmatrix}
1 & 50 & 600 & 0.02
\end{bmatrix}
$$

となります。先頭に $1$ があるのは、Step 1 で $x_0 = 1$ と定めたためです。

## パラメータを最適化する

それでは、目的関数 $L$ を最小化するモデルのパラメータベクトル ${\bf w}$ を求めましょう。
単回帰分析と同様に、目的関数をパラメータで微分して 0 とおき、${\bf w}$ について解きます。

まずは目的関数に登場している予測値 ${\bf y}$ を、パラメータ ${\bf w}$ を用いた表記に置き換えます。

$$
\begin{aligned}
L
&= ({\bf t} - {\bf y})^{\rm T} ({\bf t} - {\bf y}) \\
&= ({\bf t} - {\bf X}{\bf w})^{\rm T} ({\bf t} - {\bf X}{\bf w}) \\
&= \{ {\bf t}^{\rm T} - ({\bf X}{\bf w})^{\rm T} \} ({\bf t} - {\bf X}{\bf w}) \\
&= ({\bf t}^{\rm T} - {\bf w}^{\rm T}{\bf X}^{\rm T}) ({\bf t} - {\bf X}{\bf w})
\end{aligned}
$$

ここで、転置の公式 $({\bf A}{\bf B})^{\rm T} = {\bf B}^{\rm T}{\bf A}^{\rm T}$ を用いました。

さらに分配法則を使って展開すると、

$$
L
= {\bf t}^{\rm T}{\bf t}
- {\bf t}^{\rm T}{\bf X}{\bf w}
- {\bf w}^{\rm T}{\bf X}^{\rm T}{\bf t}
+ {\bf w}^{\rm T}{\bf X}^{\rm T}{\bf X}{\bf w}
$$

となります。この目的関数に対しパラメータの ${\bf w}$ についての偏微分を計算します。

その前に、この式をもう少し整理します。
まず、$(1)^{\rm T} = 1$ のように、スカラーは転置しても変化しません。
また、${\bf w} \in \mathbb{R}^{M+1}$、${\bf X} \in \mathbb{R}^{N \times (M+1)}$ であり、${\bf X}{\bf w} \in \mathbb{R}^{N}$ となることから、${\bf t} \in \mathbb{R}^{N}$ との間の内積 ${\bf t}^{\rm T}{\bf X}{\bf w} \in \mathbb{R}$ は、スカラーになります。
したがって、

$$
({\bf t}^{\rm T}{\bf X}{\bf w})^{\rm T} = {\bf t}^{\rm T}{\bf X}{\bf w}
$$

が成り立ちます。

さらに、転置の公式 $({\bf A}{\bf B}{\bf C})^{\rm T} = {\bf C}^{\rm T}{\bf B}^{\rm T}{\bf A}^{\rm T}$ より、

$$
({\bf t}^{\rm T}{\bf X}{\bf w})^{\rm T} = {\bf w}^{\rm T} {\bf X}^{\rm T} {\bf t}
$$

も成り立ちます。以上から、

$$({\bf t}^{T}{\bf X}{\bf w})^{T} = {\bf t}^{T}{\bf X}{\bf w} = {\bf w}^{T} {\bf X}^{T} {\bf t}$$

が導かれます。目的関数 $\mathcal{L}$ は、この式を利用して、

$$
L = {\bf t}^{\rm T}{\bf t} - 2{\bf t}^{\rm T}{\bf X}{\bf w}+ {\bf w}^{\rm T}{\bf X}^{\rm T}{\bf X}{\bf w}
$$

と変形できます。

ここで、${\bf w}$ に関する偏微分を行いやすくするため、${\bf w}$ 以外の定数項を一つにまとめます。

$$
\begin{aligned}
L
&= {\bf t}^{\rm T}{\bf t}
- 2{\bf t}^{\rm T}{\bf X}{\bf w}
+ {\bf w}^{\rm T}{\bf X}^{\rm T}{\bf X}{\bf w} \\
&= {\bf t}^{\rm T}{\bf t}
- 2({\bf X}^{\rm T}{\bf t})^{\rm T} {\bf w}
+ {\bf w}^{\rm T}{\bf X}^{\rm T}{\bf X}{\bf w} \\
&= c + {\bf b}^{\rm T}{\bf w} + {\bf w}^{\rm T}{\bf A}{\bf w}
\end{aligned}
$$

すると、${\bf w}$ に関する二次形式で表現できました。
ここで、

$$
\begin{align}
{\bf A} &= {\bf X}^{\rm T}{\bf X} \\
{\bf b} &= -2 {\bf X}^{\rm T}{\bf t} \\
c &= {\bf t}^{\rm T}{\bf t}
\end{align}
$$

とおいていることに注意してください。

それでは、目的関数を最小にするパラメータ ${\bf w}$ の求め方を考えます。
目的関数はパラメータ ${\bf w}$ に関して二次関数になっています。
まずは、${\bf w}$ 以外のベクトルや行列に、具体的な数値を当てはめて考えてみましょう。
例えば、

$$
{\bf w} =
\begin{bmatrix}
w_1 \\ w_2
\end{bmatrix}, 
{\bf A} = 
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix},
{\bf b} =
\begin{bmatrix}
1 \\
2
\end{bmatrix},
c = 1 
$$ 

とおきます。すると、目的関数の値は

$$
\begin{aligned} 
L
&= {\bf w}^{\rm T}{\bf A}{\bf w} +{\bf b}^{\rm T}{\bf w} + c \\
&=
\begin{bmatrix}
w_1 & w_2
\end{bmatrix}
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2
\end{bmatrix}
+
\begin{bmatrix}
1 & 2
\end{bmatrix}
\begin{bmatrix} 
w_1 \\ 
w_2 
\end{bmatrix}
+
1 \\ 
&=
\begin{bmatrix} 
w_1 & w_2
\end{bmatrix} 
\begin{bmatrix} 
w_1 + 2w_2 \\ 
3w_1 + 4w_2
\end{bmatrix}
+ w_1 + 2w_2 + 1 \\
&= w_1 (w_1 + 2w_2) + w_2 (3w_1 + 4w_2) + w_1 + 2w_2 + 1 \\ 
&= w_1^2 + 5 w_1 w_2 + 4 w_2^2 + w_1 + 2 w_2 + 1 \\
\end{aligned}
$$

となります。これを $w_1, w_2$ に関して整理すると、

$$
\begin{aligned}
L
&= w_1^2 + (5 w_2 + 1) w_1 + (4 w_2^2 + 2 w_2 + 1) \\
&= 4 w_2^2 + (5 w_1 + 2) w_2 + (w_1^2 + w_1 + 1)
\end{aligned} 
$$

となり、$w_1, w_2$ それぞれについて二次関数になっていることが分かります。

目的関数 $L$ を $w_1$ の二次関数、$w_2$ の二次関数と見たとき、$L$ は、下図のような概形となっています。

![01](http://drive.google.com/uc?export=view&id=1dz01fXQXsO0FVkTkTvye3BROe5alf2eS)

さらに、各次元が $w_1, w_2, L$ を表す 3 次元空間上においては、 $L$ の概形は下図のようになっています。

![02](http://drive.google.com/uc?export=view&id=19B6p25z0-CxeOe9nvLX-lm9jnl_10lBx)

ここでは 2 つのパラメータ $w_1$ と $w_2$ について図示していますが、目的関数が 任意の $M$ 個の変数 $w_0, w_1, w_2, \dots, w_M$ によって特徴づけられている場合でも、目的関数がそれぞれのパラメータについて二次形式になっている限り、同様のことが言えます。
すなわち、$M + 1$ 個の連立方程式、

$$
\begin{cases}
\frac {\partial }{\partial w_0}L = 0 \\
\frac {\partial }{\partial w_1}L = 0 \\
\ \ \ \ \ \vdots \\
\frac {\partial }{\partial w_M}L = 0 \\
\end{cases}
$$

を解けば良いということになります。
これはベクトルによる微分を用いて表記すると、以下のようになります。

$$
\begin{aligned}
\begin{bmatrix}
\frac {\partial}{\partial w_0} L \\
\frac {\partial}{\partial w_1} L \\
\vdots  \\
\frac {\partial}{\partial w_M} L \\
\end{bmatrix}
&=
\begin{bmatrix}
0 \\
0 \\
\vdots  \\
0 \\
\end{bmatrix} \\
\Rightarrow \frac {\partial}{\partial {\bf w}} L
&= {\bf 0} \\
\end{aligned}
$$

上式を ${\bf w}$ について解くために、以下のような式変形を行います。
式変形の途中で理解できない部分があった場合は、前章の線形代数の部分を読み返してみてください。
まずは、左辺について整理を行います。
$$
\begin{aligned}
\frac{\partial}{\partial {\bf w}} L
&= \frac{\partial}{\partial {\bf w}} (c + {\bf b}^{\rm T}{\bf w} + {\bf w}^{\rm T}{\bf A}{\bf w}) \\
&=\frac{\partial}{\partial {\bf w}} (c) + \frac{\partial}{\partial {\bf w}} ({\bf b}^{\rm T}{\bf w}) + \frac{\partial}{\partial {\bf w}} ({\bf w}^{\rm T}{\bf A}{\bf w}) \\
&={\bf 0} + {\bf b} + ({\bf A} + {\bf A}^{\rm T}) {\bf w}
\end{aligned}
$$

これを $0$ とおき、${\bf A}$ 、${\bf b}$ を展開すると

$$
\begin{aligned}
-2{\bf X}^{\rm T}{\bf t} + \{ {\bf X}^{\rm T}{\bf X} + ({\bf X}^{\rm T}{\bf X})^{\rm T} \} {\bf w}
&= {\bf 0} \\
-2{\bf X}^{\rm T}{\bf t} + 2{\bf X}^{\rm T}{\bf X}{\bf w}
&= {\bf 0} \\
{\bf X}^{\rm T}{\bf X}{\bf w}& = {\bf X}^{\rm T}{\bf t} \\
\end{aligned}
$$

のように式変形できます。

ここで、${\bf X}^{\rm T}{\bf X}$に**逆行列が存在すると仮定**して、両辺に左側から $({\bf X}^{\rm T}{\bf X})^{-1}$ を掛けると、

$$
\begin{aligned}
({\bf X}^{\rm T}{\bf X})^{-1} {\bf X}^{\rm T}{\bf X} {\bf w} &= ({\bf X}^{\rm T}{\bf X})^{-1} {\bf X}^{\rm T}{\bf t} \\
{\bf I}{\bf w} &= ({\bf X}^{\rm T}{\bf X})^{-1} {\bf X}^{\rm T}{\bf t} \\
{\bf w} &= ({\bf X}^{\rm T}{\bf X})^{-1}{\bf X}^{\rm T}{\bf t}
\end{aligned}
$$

が導かれます。特に、この最後の式を**正規方程式 (normal equation)** と呼びます。  

上式は、与えられたデータを並べた行列 ${\bf X}$ と、各データの目標値を並べたベクトル ${\bf t}$ から最適なパラメータ ${\bf w}$ を計算しており、新しい入力変数の値 ${\bf x} = [x_1, \dots, x_M]^{\rm T}$ に対して予測値 $y$ を得るためには、ここで求めたパラメータ ${\bf w}$を用いて、

$$
y = {\bf w}^{\rm T}{\bf x}
$$

のように計算すると良いです。

## 練習問題 本章のまとめ
本章で学んだ内容を復習しましょう。問題の答えをそれぞれのセルに記載してください。  
（プログラミングを行うのではなく、問題をノート上などで解き、回答を空白のセルに記入してください。）

重回帰分析を用いて、パラメータ $w$ を算出してください。数値は下記を用いてください。  
（実際にはバイアス $b$ を考慮し計算を行う必要がありますが、この練習問題ではバイアスは考慮せずに計算を行ってください。）

|部屋の広さ|駅からの距離|家賃|
|:--|:--|:--|
|1|2|10|
|2|3|14|

逆行列の計算は下記を参考にしてください。  

下記のように行列 A の逆行列を求める場合は、

$$
A=\begin{pmatrix}a&b\\c&d\end{pmatrix}
$$

下記のような計算となります。

$$
A^{-1}=\dfrac{1}{ad-bc}\begin{pmatrix}d&-b\\-c&a\end{pmatrix}
$$

実際の値を入れて確認してみます。  
$a = 1$ 、$b = 2$ 、$c = 3$ 、$d = 4$ とします。

$$
A^{-1}=\dfrac{1}{1 \times 4 - 2 \times 3}\begin{pmatrix}1&2\\3&4\end{pmatrix}
$$

対角成分（この場合 1 と 4）を入れ替えます。

$$
A^{-1}=\dfrac{1}{- 2}\begin{pmatrix}4&2\\3&1\end{pmatrix}
$$

非対角成分（この場合 2 と 3）を $-1$ 倍します。


$$
A^{-1}=\dfrac{1}{- 2}\begin{pmatrix}4&-2\\-3&1\end{pmatrix}
$$

$-$ を行列内に入れます。全ての値の符号が下記のように入れ替わります。

$$
A^{-1}=\dfrac{1}{2}\begin{pmatrix}-4&2\\3&-1\end{pmatrix}
$$

これで逆行列を求めることができました。

In [2]:
# 重回帰分析のパラメータ w の値
import numpy as np

A = np.array([[1, 2],
              [2, 3]])
b = np.array([10, 14])

ATA = np.dot(A.T, A)

ATA_inv = np.linalg.inv(ATA)

ATb = np.dot(A.T, b)

w = np.dot(ATA_inv, ATb)


print("w =", w)






w = [-2.  6.]


$$
\begin{aligned}
{\bf w}
&={
    \begin{bmatrix}
    -2 \\
    6
    \end{bmatrix}
}
\end{aligned}
$$


### 模範解答

$$
{\bf X} = 
\begin{bmatrix}
1 & 2 \\
2 & 3
\end{bmatrix} ,\
{\bf t} = 
\begin{bmatrix}
10 \\
14
\end{bmatrix}
$$


重回帰分析のパラメータ $\bf{w}$ を求める式はこちらでした。  


$$
{\bf w} = ({\bf X}^{\rm T}{\bf X})^{-1}{\bf X}^{\rm T}{\bf t}
$$



上記の式に数値を当て込み計算を行います。  

$$
\begin{aligned}
{\bf w} 
&= \Biggl(\begin{bmatrix}
1 & 2 \\
2 & 3 \\
\end{bmatrix}^{\rm T}
\begin{bmatrix}
1 & 2 \\
2 & 3 \\
\end{bmatrix}
\Biggl)^{-1}
\begin{bmatrix}
1 & 2 \\
2 & 3 \\
\end{bmatrix}^{\rm T}
\begin{bmatrix}
10 \\
14 \\
\end{bmatrix} \\
\end{aligned}
$$

<img src="http://drive.google.com/uc?export=view&id=1g2xjXbw5qYeqdJqcOf3uASvzBQxhlE8u" width=30%>